# Методы сбора и обработки данных из сети Интернет
## Open data
#### 1.	Создать приложение, которое будет из готового файла с данными «Сбербанка» (https://www.sberbank.com/ru/analytics/opendata) выводить результат по параметрам:
* Тип данных
* Интервал дат
* Область

#### 2.	Визуализировать выводимые данные с помощью графика

### Использование IPywidgets
#### 1. Для локального просмотра:
Для интерактивного просмотра используется **IPywidgets**: 

`pip install ipywidgets`

Далее для активации IPywidgets в Jupyter Notebook выполняем:

`jupyter nbextension enable --py widgetsnbextension`

А для использования в JupyterLab:

`jupyter labextension install @jupyter-widgets/jupyterlab-manager`

### Практическая работа



In [1]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

Создаем DataFrame, конвертируем строку с датой в Datetime.

In [2]:
df = pd.read_csv('http://sbrfdata.ru/opendata.zip', engine='python')
df.date = pd.to_datetime(df.date, format="%Y-%m-%d")

Строим списки уникальных значений регионов и метрик.

In [3]:
metrics = df.name.unique()
regions = df.region.unique()

И сам динамический график

In [4]:
def draw_plot(metrics, region, start_date, end_date):
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    df_plot = df[(df.name == metrics) & 
             (df.region == region) & 
             (df.date >= start_date) &
             (df.date <= end_date)]
    
    x = list(df_plot.date)
    y = list(df_plot.value)
    
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.plot_date(x, y, linestyle='-')
    
    for i in range(len(x)):
        ax.annotate(y[i], (mdates.date2num(x[i]), y[i]), ha="right", va="center",
                    xytext=(20, 15), textcoords='offset points', bbox=dict(boxstyle="round", fc="w", ec="k"))
    
    plt.show()
    
_ = interact(draw_plot, 
             metrics=widgets.Dropdown(options=list(metrics)),
             region=widgets.Dropdown(options=list(regions)), 
             start_date = widgets.DatePicker(value=pd.to_datetime('2018-01-01')), 
             end_date = widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

interactive(children=(Dropdown(description='metrics', options=('Количество заявок на потребительские кредиты',…